# CAO Points Analysis

[CAO](http://www.cao.ie/index.php?page=points&p=2021)

***

In [1]:
# Data Frames
import pandas as pd
from collections import namedtuple
import requests, PyPDF2
# Regular Expressions
import re

# Convenient HTTP Requests.
import requests as rq

# dates and times
import datetime as dt

# For downloading
import urllib.request as urlrq

In [2]:
# Fetch CAO points URL
resp = rq.get('http://www2.cao.ie/points/l8.php')
# Have a quick peek
resp

<Response [200]>

# Save original data set

In [3]:
# Get Current date and time.
now = dt.datetime.now()

# Format as string
nowstr = now.strftime('%Y%m%d_%H%M%S')

In [4]:
#Create file path for original data
pathhtml = 'data/cao2021_'+ nowstr + '.html'

### Error on Server
- Server says we should decode as per:
    Content-Type: text/html; charset=iso-8859-1
- However one line uses \x96 which is not defined in iso-8859-1
- Therefore we used the similar decoding standard cp1252
- This is very similar but includes \x96

In [5]:
# Server uses the incorrect encoding
original_encoding = resp.encoding
# Change to cp1252
resp.encoding = 'cp1252'

In [6]:
# Save The orginal html file
with open(pathhtml, 'w') as f:
    f.write(resp.text)

In [7]:
resp.encoding

'cp1252'

# Use regular Expressions to select the lines we want

In [8]:
# Compile Reg-Ex for Matching lines

re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

def points_to_array(s):
    portfolio = ''
    if s[0] == '#':
        portfolio = '#'
    random = ''
    if s[-1] == '*':
        random = 'True'
    points = ''
    for i in s:
        if i.isdigit():
            points = points + i
    return [points, portfolio, random]

In [9]:
# File path for data 
path2021 = 'data/cao2021_csv_'+ nowstr + '.csv'
    

In [10]:


# Keep track of how many courses we process.
no_lines = 0

# Open CSV file for writing 
with open (path2021, 'w') as f:
    # Write a header row
    f.write(','.join(['Code', 'Title', 'PointsRD1', 'PointsRD2']) + '\n')
    # loop through lines of the response
    for line in resp.iter_lines():
        #decode the line using 'cp1252' - wrong encoding
        dline = line.decode('cp1252')
        # Match only the lines representing courses.
        if re_course.fullmatch(dline):
            # Add one to the lines counter.
            no_lines = no_lines + 1
            # The course code
            course_code = dline[:5]
            #print (course_code)
            # course title
            course_title = dline[7:57]
            #print (course_title)
            # Round one points
            course_points = re.split(' +', dline[60:])
            if len(course_points) !=2:
                course_points = course_points[ :2]
            # join the fields using a comma
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas
            f.write(','.join(linesplit) + '\n')

# Print the total number of processed lines.
print(f"Total number of lines is {no_lines}.")

Total number of lines is 949.


<br>

**NB:** it was verified as of 09/11/2021 that there were 949 courses exactly in the cao2021 points list

In [11]:
df2021 = pd.read_csv(path2021)

In [12]:
df2021

,Code,Title,PointsRD1,PointsRD2
0,AL801,Software Design for Virtual Reality and Gaming...,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructur...,321,NaN
4,AL810,Quantity Surveying ...,328,NaN
...,...,...,...,...
944,WD211,Creative Computing ...,270,NaN
945,WD212,Recreation and Sport Management ...,262,NaN
946,WD230,Mechanical and Manufacturing Engineering ...,230,230
947,WD231,Early Childhood Care and Education ...,266,NaN


<br>

## 2020 CAO Points

In [13]:
# Data Frames
import pandas as pd
from collections import namedtuple
import requests, PyPDF2
# Regular Expressions
import re

# Convenient HTTP Requests.
import requests as rq

# dates and times
import datetime as dt

# For downloading
import urllib.request as urlrq

##### Steps to reproduce

1. Download original PDF
2. Open PDF using Microsoft Word
3. Save MS Words convered PDF in docx format
4. Re-save Word Doc for editing
5. Delete Headers and footers
6. Delete preamble and intro on first page
7. Select all and copy
8. Paste into VS Code
9. Remove HEI Name lines and black lines
10. Change column heading "Course and intitution" to COURSE
11. Change Backticks to apostrophes
12. Replace double tab cahracter on line 28 with single tab
13. Delete tabs at end of lines 604, 670, 700, 701, 793 and 830

In [14]:
url2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

In [15]:
# Get Current date and time.
now = dt.datetime.now()

# Format as string
nowstr = now.strftime('%Y%m%d_%H%M%S')

In [16]:
#Create file path for original data
pathxlsx = 'data/cao2020_'+ nowstr + '.xlsx'

In [17]:
# Save original file to disk
urlrq.urlretrieve(url2020, pathxlsx)

('data/cao2020_20211109_224116.xlsx',
 <http.client.HTTPMessage at 0x21fce502e80>)

<br> 

## Load Spreadsheet using Pandas

In [18]:
# Download and parse the excel spreadsheet
df2020 = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', skiprows = 10)

In [19]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Spot Check Random Row
df2020.iloc[753]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Road Transport Technology and Management
COURSE CODE2                                                           LC286
R1 POINTS                                                                264
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      264
EOS Random *                                                             NaN
EOS Mid-point                                                            360
LEVEL                                                                      7
HEI                                         Limerick Institute of Technology
Test/Interview #                                                         NaN

In [21]:
# Spot Check End of Data
df2020.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [22]:
#Create file path for PANDAS data
path = 'data/cao2020_'+ nowstr + '.csv'

In [23]:
# Save pandas data frame to disk
df2020.to_csv(path)

# 2019 Points

In [24]:
df2019A = pd.read_csv('data/2019POINTS_20211104103000_edited.csv', sep= '\t')

In [25]:
df2019 = df2019A.loc[:, ~df2019A.columns.str.contains('^Unnamed')]

In [26]:
df2019A

,HEI,Course Code,INSTITUTION and COURSE,EOS,Mid,Unnamed: 5
0,Athlone Institute of Technology,AL801,Software Design with Virtual Reality and Gaming,304,328.0,NaN
1,Athlone Institute of Technology,AL802,Software Design with Cloud Computing,301,306.0,NaN
2,Athlone Institute of Technology,AL803,Software Design with Mobile Apps and Connected...,309,337.0,NaN
3,Athlone Institute of Technology,AL805,Network Management and Cloud Infrastructure,329,442.0,NaN
4,Athlone Institute of Technology,AL810,Quantity Surveying,307,349.0,NaN
...,...,...,...,...,...,...
925,Waterford Institute of Technology,WD200,Arts (options),221,296.0,NaN
926,Waterford Institute of Technology,WD210,Software Systems Development,271,329.0,NaN
927,Waterford Institute of Technology,WD211,Creative Computing,275,322.0,NaN
928,Waterford Institute of Technology,WD212,Recreation and Sport Management,274,311.0,NaN


<br>

## Concatonate and Join

<br>

Concatonate and Join

In [30]:
points2021 = df2021[['Code', 'Title']]
points2021

,Code,Title
0,AL801,Software Design for Virtual Reality and Gaming...
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructur...
4,AL810,Quantity Surveying ...
...,...,...
944,WD211,Creative Computing ...
945,WD212,Recreation and Sport Management ...
946,WD230,Mechanical and Manufacturing Engineering ...
947,WD231,Early Childhood Care and Education ...


In [28]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df2019

,HEI,Course Code,INSTITUTION and COURSE,EOS,Mid
0,Athlone Institute of Technology,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,Athlone Institute of Technology,AL802,Software Design with Cloud Computing,301,306.0
2,Athlone Institute of Technology,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,Athlone Institute of Technology,AL805,Network Management and Cloud Infrastructure,329,442.0
4,Athlone Institute of Technology,AL810,Quantity Surveying,307,349.0
...,...,...,...,...,...
925,Waterford Institute of Technology,WD200,Arts (options),221,296.0
926,Waterford Institute of Technology,WD210,Software Systems Development,271,329.0
927,Waterford Institute of Technology,WD211,Creative Computing,275,322.0
928,Waterford Institute of Technology,WD212,Recreation and Sport Management,274,311.0
